In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import plotly_express as px
from fastai.vision import *
import pandas as pd

In [27]:
test_path = Path('dataset_classification_final_test')
model_path = Path('dataset_classification/')

In [5]:
#load learner exported using learn.export()
learn = load_learner(model_path, test=ImageList.from_folder(test_path))

In [ ]:
Prediction = namedtuple('Prediction', ['path', 'pred_idx', 'pred_class', 'probability', 'misclassified'])
def test_model_class(folder):   
    result = []    
    files_list = [files for files in folder.iterdir() if files.suffix == '.png']
    for f in files_list:
        misclass = False
        img = open_image(f)
        pred_class,pred_idx,outputs = learn.predict(img)
        if not (str(pred_class) == folder.name):
            print(f"Error! predicted class: '{pred_class}' vs actual class: '{folder.name}'")
            print("\t",f)
            misclass = True
        result.append((f, int(pred_idx), str(pred_class), outputs, misclass))
        
    return result

In [29]:
def get_results_class(folder_class):
    result = []    
    files_list = get_image_files(folder_class)
    for f in files_list:
        misclass = False
        actual_class = folder_class.name
        img = open_image(f)
        pred_class,pred_idx,outputs = learn.predict(img)
        if not (str(pred_class) == actual_class):
            print(f"Error! predicted class: '{pred_class}' vs actual class: '{actual_class}'")
            print("\t",f)
            misclass = True
# tuple legend: path, actual_class, pred_class, misclass, pred_idx, prob_all_classes        
        result.append((str(f), actual_class, str(pred_class), misclass, int(pred_idx), outputs))
        
    return result

In [ ]:
get_results_class(test_path / 'Albicocca')

In [46]:
def get_results():
    folders_list = sorted([folders for folders in test_path.iterdir()])
    results = pd.DataFrame(columns=['path', 'actual_class', 'pred_class', 'misclass', 'pred_idx', 'prob_all_classes'])
    for f in folders_list:
        result = get_results_class(f)
        result = pd.DataFrame(result, columns=['path', 'actual_class', 'pred_class', 'misclass', 'pred_idx', 'prob_all_classes'])
        results = pd.concat((results, result))
    return results

In [47]:
get_results()

Error! predicted class: 'Prugna rossa' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_21.png
Error! predicted class: 'Prugna rossa' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_87.png
Error! predicted class: 'Patata' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_79.png
Error! predicted class: 'Patata' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_71.png
Error! predicted class: 'Patata' vs actual class: 'Pesca'
	 dataset_classification_final_test/Pesca/Pesca_54.png


,path,actual_class,pred_class,misclass,pred_idx,prob_all_classes
0,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9968), tensor(4.2674e-05), tensor(0...."
1,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9963), tensor(6.5193e-05), tensor(0...."
2,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9943), tensor(0.0002), tensor(0.0014..."
3,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9941), tensor(6.1927e-05), tensor(0...."
4,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9956), tensor(7.4738e-05), tensor(0...."
5,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9888), tensor(0.0002), tensor(0.0038..."
6,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9969), tensor(6.7007e-05), tensor(0...."
7,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9702), tensor(0.0010), tensor(0.0081..."
8,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9894), tensor(7.2631e-05), tensor(0...."
9,dataset_classification_final_test/Albicocca/Al...,Albicocca,Albicocca,False,0,"[tensor(0.9980), tensor(2.0273e-05), tensor(9...."


In [48]:
results = _

In [54]:
results.prob_all_classes[0]

RuntimeError: bool value of Tensor with more than one value is ambiguous